In [1]:
import json
import requests
import websockets
import sys
from typing import Tuple, List
from math import log
inf = sys.maxsize

# Global Variables and Functions

In [2]:
# dictionary stored data in a currency: currency exchange rate format
rates = [
                    {
                        "ETH": {"ETH": 1, "BTC": inf,"XMR": inf},
                        "BTC": {"ETH": inf,"BTC": 1,"XMR": inf},
                        "XMR": {"ETH": inf,"BTC": inf, "XMR": 1}
                    }
]

currencies = (
    'ETH', 
    'BTC', 
    'XMR'
)

def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [[-log(edge) for edge in row.values()] for row in graph[0].values()]
    return result


def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get a result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # Relax edges |V-1| times
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                    pre[dest_curr] = source_curr

    # if we can still relax edges, then we have a negative cycle = arbitrage cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                # negative cycle exists, and use the predecessor chain to print the cycle
                print_cycle = [dest_curr, source_curr]
                # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                while pre[source_curr] not in  print_cycle:
                    print_cycle.append(pre[source_curr])
                    source_curr = pre[source_curr]
                print_cycle.append(pre[source_curr])
                print("Arbitrage Opportunity: \n")
                print(" --> ".join([currencies[p] for p in print_cycle[::-1]]))

def kelly_criterion(balance: float, expReturn: float, probWin: float, probLose: float):
    kelly = probWin - (probLose / expReturn)
    optimalStake = round(balance * kelly, 2)
    print(f"Kelly Percentage is: {round(kelly * 100, 2)}%")
    print(f"Optimal Stake is: ${optimalStake}")
    return kelly

def expected_return(oppurtunity):
    starting_amount = 1
    profit = 1 # sets the value of the starting crypto currency ammount
    count = [len(oppurtunity) - 1] # used to avoid out of index range error
    print(f"Starting With {profit} {oppurtunity[0]}(s)")
    
    for i in range(len(oppurtunity)): # loops thru each currency
        
        currRates = rates[0][oppurtunity[i]] # saves the current itration currecny rates
        
        if i in count: # check if it is the last iteration
            
            curr = oppurtunity[i]
            profit = profit * currRates[curr] # multiply total by 1
            
        else:
            
            curr = oppurtunity[i + 1]
            profit = profit * currRates[curr] # multiply total by exchange rate
            net_profit = profit - starting_amount
            
    expected_return = (net_profit / profit)
            
    print(f"You End Up With {profit} {oppurtunity[0]}(s)")
    print(f"With A {expected_return}% Return")
    return expected_return

def profit_calculator(oppurtunity, investment):
    starting_amount = investment
    profit = investment # sets the value of the starting crypto currency ammount
    count = [len(oppurtunity) - 1] # used to avoid out of index range error
    print(f"Starting With {profit} {oppurtunity[0]}(s)")
    
    for i in range(len(oppurtunity)): # loops thru each currency
        
        currRates = rates[0][oppurtunity[i]] # saves the current itration currecny rates
        
        if i in count: # check if it is the last iteration
            
            curr = oppurtunity[i]
            profit = profit * currRates[curr] # multiply total by 1
            
        else:
            
            curr = oppurtunity[i + 1]
            profit = profit * currRates[curr] # multiply total by exchange rate
            
    print(f"You End Up With {profit} {oppurtunity[0]}(s)")
    return profit

# Apply public connect token

In [3]:
url = 'https://api.kucoin.com/api/v1/bullet-public'

In [4]:
x = requests.post(url)

In [5]:
cjson = json.loads(x.text)

In [6]:
endpoint = cjson['data']['instanceServers'][0]['endpoint']
token = cjson['data']['token']

# Multiplex Websocket Connection

In [7]:
multi = {"id": "1Jpg30DEdU", "type": "openTunnel", "newTunnelId": "bt1", "response": False}
# command used to open tunnel

In [8]:
multi_ETHBTC = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:ETH-BTC",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [9]:
multi_XMRBTC = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:XMR-BTC",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [10]:
multi_XMRETH = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:XMR-ETH",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [11]:
async with websockets.connect(endpoint+"?token="+token) as conn:
    await conn.send(json.dumps(multi))  # initial open tunel command
    res = await conn.recv()
    
    await conn.send(json.dumps(multi_ETHBTC)) # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_XMRBTC))  # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_XMRETH))  # send tunnel subscription
    res = await conn.recv()
    
    counter = 0
    spotPrice = 0
    futurePrice = 0
    while True and counter < 10:
        
        res = await conn.recv()
        vals = json.loads(res)

        if (vals['topic']) == '/market/ticker:ETH-BTC':
            rates[0]['ETH']['BTC'] = 1 / float(vals['data']['bestAsk']) # takes the inverse of the exchange rate to get the reverse
            rates[0]['BTC']['ETH'] = float(vals['data']['bestAsk']) # saves the exchanges exchange rate to designated variable

        elif (vals['topic']) == '/market/ticker:XMR-ETH':
            rates[0]['XMR']['ETH'] = 1 / float(vals['data']['bestAsk'])
            rates[0]['ETH']['XMR'] = float(vals['data']['bestAsk'])
        
        elif (vals['topic']) == '/market/ticker:XMR-BTC':
            rates[0]['XMR']['BTC'] = 1 / float(vals['data']['bestAsk'])
            rates[0]['BTC']['XMR'] = float(vals['data']['bestAsk'])
        
        
        if __name__ == "__main__":
            arbitrage(currencies, rates)
        counter += 1
    eth_tunnel = {"id": "1JpoPamgFM", "type": "closeTunnel", "tunnelId": "bt1", "response": False}            
    btc_tunnel = {"id": "1JpoPamgFN", "type": "closeTunnel", "tunnelId": "bt1", "response": False}
    await conn.send(json.dumps(eth_tunnel))
    await conn.send(json.dumps(btc_tunnel))
    await conn.close()

Arbitrage Opportunity: 

BTC --> ETH --> BTC
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

BTC --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

BTC --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC


In [13]:
expected_return(['BTC', 'XMR', 'ETH', 'BTC'])

Starting With 1 BTC(s)
You End Up With 1.0007094733122353 BTC(s)
With A 0.0007089703167163764% Return


0.0007089703167163764

In [15]:
profit_calculator(['BTC', 'XMR', 'ETH', 'BTC'], 100)

Starting With 100 BTC(s)
You End Up With 100.07094733122352 BTC(s)


100.07094733122352

In [16]:
kelly_criterion(20000, 4.04,0.59,0.41)

Kelly Percentage is: 48.85%
Optimal Stake is: $9770.3


0.4885148514851485

In [17]:
kelly_criterion(1000000, 0.31980194294926867 * 10, 0.59, 0.41)

Kelly Percentage is: 46.18%
Optimal Stake is: $461795.65


0.4617956507021473

In [22]:
# simple simulation account
class portfolio:
    
    # set the balance of the portfolio
    def __init__(self, starting_balance):
        self.balance = starting_balance
    
    # add funds to the account
    def fund(self, funding_amount):
        self.balance += funding_amount
    
    # withdraw funds from the account
    def withdraw(self, withdrawlamount):
        self.balance -= withdrawlamount
     
    # execute arbitrage stratedgy on the acccount using kelly criterion stake
    def arbitrage_with_kelly_stake(self):
        kelly = kelly_criterion(self.balance, expected_return(['BTC', 'XMR', 'ETH', 'BTC']) * 10, 0.59, 0.41)
        stake = self.balance * kelly
        self.balance -= stake
        profit = profit_calculator(['BTC', 'XMR', 'ETH', 'BTC'], stake)
        self.balance += profit
        
    
    # execute arbitrage stratedgt using a custom stake
    def arbitrage_with_custom_stake(self, stake_amount):
        self.balance -= stake_amount
        profit = profit_calculator(['BTC', 'XMR', 'ETH', 'BTC'], stake_amount)
        self.balance += profit

In [23]:
myPortfolio = portfolio(1000000)

In [24]:
# assuming your balance is all one cryptocurrency
myPortfolio.balance

1000000

In [25]:
myPortfolio.arbitrage_with_kelly_stake()

Starting With 1 BTC(s)
You End Up With 1.0007094733122353 BTC(s)
With A 0.0007089703167163764% Return
Kelly Percentage is: -5724.03%
Optimal Stake is: $-57240347.81
Starting With -57240347.80622847 BTC(s)
You End Up With -57280958.30538005 BTC(s)


In [26]:
myPortfolio.balance

959389.50084842